# DVS128 Event Camera 3D Visualization

Interactive 3D visualization of event camera data: **Time × X × Y**

Similar to MATLAB's `PlotPoint3D` but for DVS events with polarity color-coding.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pathlib import Path

# Enable interactive 3D plots
%matplotlib widget
# If that doesn't work, use: %matplotlib notebook
# Or for static plots: %matplotlib inline

plt.rcParams['figure.figsize'] = (14, 10)

## 1. Load Your CSV File

In [ ]:
# ========================================
# CONFIGURE YOUR CSV PATH HERE
# ========================================

CSV_PATH = "~/Desktop/QuantumNetwork/CSVs/sine_300mV.csv"  # ← CHANGE THIS!

# Time window to visualize (in seconds)
TIME_START = 0.0
TIME_END = 2.0     # Visualize first 2 seconds

# Limit events for performance
MAX_EVENTS = 15000  # Reduce if visualization is slow

In [ ]:
def load_events(csv_path, max_events=10000, time_window=None):
    """
    Load events from CSV file.
    
    Returns:
        DataFrame with columns [timestamp_s, x, y, polarity]
    """
    csv_path = Path(csv_path).expanduser()
    
    if not csv_path.exists():
        raise FileNotFoundError(f"CSV not found: {csv_path}")
    
    print(f"Loading: {csv_path.name}")
    
    # Read CSV
    df = pd.read_csv(csv_path)
    
    # Standardize columns
    df.columns = df.columns.str.strip().str.lower()
    
    # Rename if needed
    if 'timestamp' in df.columns and 'timestamp_us' not in df.columns:
        df.rename(columns={'timestamp': 'timestamp_us'}, inplace=True)
    
    # Convert to seconds
    df['timestamp_s'] = df['timestamp_us'] / 1e6
    df['timestamp_s'] = df['timestamp_s'] - df['timestamp_s'].min()  # Start at 0
    
    # Map polarity to 0/1
    if df['polarity'].min() < 0:
        df['polarity'] = (df['polarity'] + 1) // 2
    
    # Filter by time window
    if time_window:
        start, end = time_window
        df = df[(df['timestamp_s'] >= start) & (df['timestamp_s'] < end)]
    
    # Sample events
    if len(df) > max_events:
        indices = np.linspace(0, len(df)-1, max_events, dtype=int)
        df = df.iloc[indices]
    
    print(f"  Events: {len(df):,}")
    print(f"  Time range: {df['timestamp_s'].min():.3f} to {df['timestamp_s'].max():.3f}s")
    print(f"  Spatial: X=[{df['x'].min()}, {df['x'].max()}], Y=[{df['y'].min()}, {df['y'].max()}]")
    
    return df[['timestamp_s', 'x', 'y', 'polarity']]

In [ ]:
# Load the data
events = load_events(
    CSV_PATH,
    max_events=MAX_EVENTS,
    time_window=(TIME_START, TIME_END)
)

print(f"\nLoaded {len(events):,} events for visualization")

## 2. 3D Plot: Time × X × Y

This is the main 3D visualization showing how events evolve over time in space.

In [ ]:
# Separate ON and OFF events
on_events = events[events['polarity'] == 1]
off_events = events[events['polarity'] == 0]

print(f"ON events (polarity=1): {len(on_events):,}")
print(f"OFF events (polarity=0): {len(off_events):,}")

In [ ]:
# Create 3D plot
fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')

# Plot ON events (blue)
if len(on_events) > 0:
    ax.scatter(on_events['timestamp_s'],
               on_events['x'],
               on_events['y'],
               c='blue',
               marker='o',
               s=2,
               alpha=0.5,
               label='ON events')

# Plot OFF events (red)
if len(off_events) > 0:
    ax.scatter(off_events['timestamp_s'],
               off_events['x'],
               off_events['y'],
               c='red',
               marker='o',
               s=2,
               alpha=0.5,
               label='OFF events')

ax.set_xlabel('Time (seconds)', fontsize=12)
ax.set_ylabel('X (pixels)', fontsize=12)
ax.set_zlabel('Y (pixels)', fontsize=12)
ax.set_title('DVS128 Events: Time × X × Y', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nRotate the 3D plot with your mouse to explore different angles!")

## 3. Multiple Views (Like MATLAB PlotPoint3D)

Four different 3D perspectives in one figure.

In [ ]:
fig = plt.figure(figsize=(16, 12))
fig.suptitle('DVS128 Event Visualization - Multiple Views', fontsize=16, fontweight='bold')

# Plot 1: Time vs X vs Y
ax1 = fig.add_subplot(2, 2, 1, projection='3d')
if len(on_events) > 0:
    ax1.scatter(on_events['timestamp_s'], on_events['x'], on_events['y'],
                c='blue', marker='o', s=1, alpha=0.5, label='ON')
if len(off_events) > 0:
    ax1.scatter(off_events['timestamp_s'], off_events['x'], off_events['y'],
                c='red', marker='o', s=1, alpha=0.5, label='OFF')
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('X')
ax1.set_zlabel('Y')
ax1.set_title('Time vs X vs Y')
ax1.legend()

# Plot 2: X vs Y (spatial view)
ax2 = fig.add_subplot(2, 2, 2)
if len(on_events) > 0:
    ax2.scatter(on_events['x'], on_events['y'],
                c='blue', marker='o', s=1, alpha=0.3, label='ON')
if len(off_events) > 0:
    ax2.scatter(off_events['x'], off_events['y'],
                c='red', marker='o', s=1, alpha=0.3, label='OFF')
ax2.set_xlabel('X (pixels)')
ax2.set_ylabel('Y (pixels)')
ax2.set_title('Spatial View (X vs Y)')
ax2.set_aspect('equal')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Time vs X
ax3 = fig.add_subplot(2, 2, 3)
if len(on_events) > 0:
    ax3.scatter(on_events['timestamp_s'], on_events['x'],
                c='blue', marker='o', s=1, alpha=0.3, label='ON')
if len(off_events) > 0:
    ax3.scatter(off_events['timestamp_s'], off_events['x'],
                c='red', marker='o', s=1, alpha=0.3, label='OFF')
ax3.set_xlabel('Time (s)')
ax3.set_ylabel('X (pixels)')
ax3.set_title('Time vs X Projection')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Time vs Y
ax4 = fig.add_subplot(2, 2, 4)
if len(on_events) > 0:
    ax4.scatter(on_events['timestamp_s'], on_events['y'],
                c='blue', marker='o', s=1, alpha=0.3, label='ON')
if len(off_events) > 0:
    ax4.scatter(off_events['timestamp_s'], off_events['y'],
                c='red', marker='o', s=1, alpha=0.3, label='OFF')
ax4.set_xlabel('Time (s)')
ax4.set_ylabel('Y (pixels)')
ax4.set_title('Time vs Y Projection')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Event Density Heatmap

In [ ]:
# Create 2D histogram (Time vs X)
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Time vs X density
H1, xedges, yedges = np.histogram2d(
    events['timestamp_s'], events['x'],
    bins=[50, 32]
)
im1 = axes[0].imshow(H1.T, aspect='auto', origin='lower', cmap='hot', interpolation='nearest')
axes[0].set_xlabel('Time bins')
axes[0].set_ylabel('X bins')
axes[0].set_title('Event Density: Time vs X')
plt.colorbar(im1, ax=axes[0])

# Time vs Y density
H2, xedges, yedges = np.histogram2d(
    events['timestamp_s'], events['y'],
    bins=[50, 32]
)
im2 = axes[1].imshow(H2.T, aspect='auto', origin='lower', cmap='hot', interpolation='nearest')
axes[1].set_xlabel('Time bins')
axes[1].set_ylabel('Y bins')
axes[1].set_title('Event Density: Time vs Y')
plt.colorbar(im2, ax=axes[1])

# X vs Y density
H3, xedges, yedges = np.histogram2d(
    events['x'], events['y'],
    bins=[32, 32]
)
im3 = axes[2].imshow(H3.T, aspect='equal', origin='lower', cmap='hot', interpolation='nearest')
axes[2].set_xlabel('X bins')
axes[2].set_ylabel('Y bins')
axes[2].set_title('Event Density: X vs Y')
plt.colorbar(im3, ax=axes[2])

plt.tight_layout()
plt.show()

## 5. Temporal Animation (Spatial Snapshots)

Show how the spatial pattern changes over time.

In [ ]:
# Divide time into slices
num_slices = 6
t_min = events['timestamp_s'].min()
t_max = events['timestamp_s'].max()
time_edges = np.linspace(t_min, t_max, num_slices + 1)

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i in range(num_slices):
    t_start = time_edges[i]
    t_end = time_edges[i + 1]
    
    # Filter events in this time slice
    slice_events = events[
        (events['timestamp_s'] >= t_start) &
        (events['timestamp_s'] < t_end)
    ]
    
    on = slice_events[slice_events['polarity'] == 1]
    off = slice_events[slice_events['polarity'] == 0]
    
    ax = axes[i]
    
    if len(on) > 0:
        ax.scatter(on['x'], on['y'], c='blue', s=2, alpha=0.5, label='ON')
    if len(off) > 0:
        ax.scatter(off['x'], off['y'], c='red', s=2, alpha=0.5, label='OFF')
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title(f't = [{t_start:.2f}, {t_end:.2f}]s')
    ax.set_xlim(0, 128)
    ax.set_ylim(0, 128)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    if i == 0:
        ax.legend()

plt.suptitle('Spatial Distribution Across Time', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. Summary Statistics

In [ ]:
print("Event Statistics:")
print(f"  Total events: {len(events):,}")
print(f"  ON events: {len(on_events):,} ({100*len(on_events)/len(events):.1f}%)")
print(f"  OFF events: {len(off_events):,} ({100*len(off_events)/len(events):.1f}%)")
print(f"\nTemporal:")
print(f"  Duration: {events['timestamp_s'].max() - events['timestamp_s'].min():.3f}s")
print(f"  Event rate: {len(events)/(events['timestamp_s'].max() - events['timestamp_s'].min()):.0f} events/s")
print(f"\nSpatial:")
print(f"  X range: [{events['x'].min()}, {events['x'].max()}]")
print(f"  Y range: [{events['y'].min()}, {events['y'].max()}]")
print(f"  Mean X: {events['x'].mean():.1f}")
print(f"  Mean Y: {events['y'].mean():.1f}")

---

## Tips

1. **Adjust `MAX_EVENTS`** if plots are slow (reduce) or you want more detail (increase)
2. **Change `TIME_START` and `TIME_END`** to visualize different portions of your data
3. **Use `%matplotlib widget`** for interactive 3D rotation (requires `ipympl`: `pip install ipympl`)
4. **Experiment with different CSV files** by changing `CSV_PATH`
5. **Save figures**: Right-click on plots and select "Save image as..."

---

**This gives you the same functionality as MATLAB's `PlotPoint3D` but for event camera data!** 🎉